In [1]:
# default_exp core

# Core

> Utility functions, classes etc used by other code in the expoco project.

In [2]:
#export
from dataclasses import dataclass
from pathlib import Path
import cv2, datetime, json, time

In [3]:
#export
def in_colab():
    "Check if the code is running in Google Colaboratory"
    try:
        return 'google.colab' in str(get_ipython())
    except:
        return False

def in_jupyter():
    "Check if the code is running in a jupyter notebook"
    try:
        return 'ZMQInteractiveShell' in str(get_ipython())
    except:
        return False

def in_notebook():
    "Check if the code is running in a jupyter notebook"
    return in_colab() or in_jupyter()

In [4]:
#export
def now():
    "Return a timestamp string that can be used in file or directory names"
    return datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')

In [5]:
#export
def path_to_str(path):
    "Format a path so that path strings can be used on different platforms"
    return str(path).replace('\\', '/')

In [6]:
#export
def get_dict_subset(d, name_prefix, remove_prefix=True, remove_additional_characters=1):
    "Return a subset of entries from a dictionary like object"
    if name_prefix is None or name_prefix == '': return d
    def _rm_prefix(k):
        if not remove_prefix: return k
        return k[len(name_prefix)+remove_additional_characters:]
    return {_rm_prefix(k): d[k] for k in d if k.startswith(name_prefix)}

A pytorch state dict will have keys like
```
[...
 'layers.0.0.weight',
 'layers.0.2.weight',
 'layers.0.2.bias',
 'layers.0.2.running_mean',
 'layers.0.2.running_var',
 'layers.0.2.num_batches_tracked',
 ...
 'layers.2.0.weight',
 'layers.2.0.bias']
```
so can get all state for a module with something like;
```
get_dict_subset(state_dict, 'layers.2.0')
```
which returns
```
{'weight': [...], 'bias': [...]}
```

In [7]:
test_dict = {}
for p1 in ['a','b','c']:
    for p2 in ['a','b','c']:
        test_dict[f'{p1}.{p2}']=f'{p1}.{p2}.value'

expected = {'a': 'b.a.value', 'b': 'b.b.value', 'c': 'b.c.value'}
assert get_dict_subset(test_dict, 'b') == expected
expected = {'b.a': 'b.a.value', 'b.b': 'b.b.value', 'b.c': 'b.c.value'}
assert get_dict_subset(test_dict, 'b', False) == expected
expected = {'.a': 'b.a.value', '.b': 'b.b.value', '.c': 'b.c.value'}
assert get_dict_subset(test_dict, 'b', True, 0) == expected

In [8]:
#export
@dataclass(frozen=True) # frozen doesn't really help (o: but we don't want to change values of this data class
class FaceLandmarks:
    "Constants to help working with facemesh landmarks"
    count=468
    top_lip_indent=0
    tip_of_nose=1
    pointer = [1, 5, 2, 218, 438] # tip_of_nose,up,down,left,right
    mouth = [0, 11, 12, 13, 14, 15, 16, 17, 18, 37, 38, 39, 40, 41, 42, 43, 57, 61, 62, 72, 73, 74, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 106, 146, 164, 165, 167, 178, 179, 180, 181, 182, 183, 184, 185, 186, 191, 204, 267, 268, 269, 270, 271, 272, 273, 287, 291, 292, 302, 303, 304, 306, 307, 308, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 324, 325, 335, 375, 391, 393, 402, 403, 404, 405, 406, 407, 408, 409, 410, 415, 424]

In [9]:
#export
def landmark_ids_to_col_names(landmark_ids, landmark_ids_to_exclude=None, coords=['x','y']):
    "Convert landmark IDs and coords into column names"
    landmark_ids, coords, col_names = sorted(landmark_ids), sorted(coords), []
    if not isinstance(landmark_ids_to_exclude, list):
        landmark_ids_to_exclude = [landmark_ids_to_exclude]
    for i in landmark_ids:
        if i in landmark_ids_to_exclude:
            continue
        for coord in coords:
            col_names.append(f'{i}{coord}')
    return col_names

In [10]:
assert ['2x', '2y'] == landmark_ids_to_col_names([1,3,2], [1,3])
assert ['1x', '1y', '1z'] == landmark_ids_to_col_names([1,2], [2], ['y','x','z'])
assert ['1x', '1y', '1z', '2x', '2y', '2z'] == landmark_ids_to_col_names([1,2], None, ['y','x','z'])

In [11]:
#export
class ImageDisplayHelper:
    "Display images in-notebook or standalone depending on runtime"
    def __init__(self, image, name='image'):
        "Create a new helper and show the image"
        self.name, self.in_notebook = name, in_notebook()
        self.show(image)
    def show(self, image):
        "Show an image"
        self.show_ipywidget(image) if self.in_notebook else self.show_cv2(image)
    def show_cv2(self, image):
        "Show an image using cv2"
        cv2.imshow(self.name, image)
        cv2.waitKey(1)
    def show_ipywidget(self, image):
        "Show an image using a jupyter widget"
        if getattr(self, 'image_widget', None) is None:
            import ipywidgets as widgets
            self.image_widget = widgets.Image(value=cv2.imencode('.png', image)[1].tobytes())
            display(self.image_widget)
        else:
            self.image_widget.value = cv2.imencode('.png', image)[1].tobytes()
    def close(self):
        "Close the image display"
        if getattr(self, 'image_widget', None) is not None:
            self.image_widget.close()
        cv2.destroyAllWindows()

In [12]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01a_camera_capture.ipynb.
Converted 10a_viseme_tabular_identify_landmarks.ipynb.
Converted 10b_viseme_tabular_data.ipynb.
Converted 10d_viseme_tabular_model.ipynb.
Converted 10e_viseme_tabular_train_model.ipynb.
Converted 10f_viseme_tabular_test_model.ipynb.
Converted 11b_viseme_image_data.ipynb.
Converted 11f_viseme_image_test_model.ipynb.
Converted 20a_gui_capture_command.ipynb.
Converted 20a_gui_main.ipynb.
Converted 70_cli.ipynb.
Converted index.ipynb.
Converted project_lifecycle.ipynb.
